In [1]:
import pandas as pd 

In [3]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [4]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [5]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}


In [6]:
dp.head()

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,DAB14570025,2025.0,38,11.102040,9774.98,108522.22,0.0102,0.2145,35812.3,0.33,0.03,4.535433,0.33,1.0,0.28,AAA,0.999
1,MOSTRADOR COTA,DAB02570025,2025.0,38,9.236233,13507.71,124760.36,0.0118,0.1619,28694.9,0.23,0.06,4.523077,0.56,1.0,0.28,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,38,8.544108,47942.77,409628.20,0.0386,0.0884,294932.3,0.72,0.09,6.260870,1.28,1.0,0.27,AAA,0.999
3,MOSTRADOR COTA,DAB02968025,2025.0,38,7.733650,20422.35,157939.31,0.0149,0.1244,52120.0,0.33,0.11,4.394366,1.61,1.0,0.29,AAA,0.999
4,MOSTRADOR COTA,BCE00606125,2025.0,38,6.681861,20640.17,137914.75,0.0130,0.1374,117227.5,0.85,0.13,11.707317,2.46,1.0,0.31,AAA,0.999
